In [1]:
from faker import Faker
import random
import uuid
import pandas as pd
fake = Faker()

In [2]:
# Initialisierung
fake = Faker("de_DE")
kompetenzen_pool = ["Machine Learning", "Data Engineering", "Statistik", "Softwarearchitektur", "NLP"]

In [3]:
# Personen erzeugen
personen = []
for _ in range(10):
    person = {
        "id": str(uuid.uuid4()),
        "name": fake.name(),
        "kompetenzen": ", ".join(random.sample(kompetenzen_pool, random.randint(1, 3))),
        "zeitbudget": round(random.uniform(0.5, 1.0), 2)
    }
    for monat in range(1, 13):
        person[f"verfuegbarkeit_{monat:02d}/2025"] = round(random.uniform(0.3, 1.0), 2)
    personen.append(person)


In [4]:
# Projekte und Teilaufgaben erzeugen
projekte = []
teilaufgaben = []

for _ in range(10):
    projekt_id = str(uuid.uuid4())
    projekt = {
        "projekt_id": projekt_id,
        "bezeichnung": fake.catch_phrase(),
        "start": "01/2025",
        "ende": "12/2025"
    }
    proj_ta = [
        {
            "teilaufgabe_id": str(uuid.uuid4()),
            "projekt_id": projekt_id,
            "bezeichnung": fake.bs(),
            "kompetenz": random.choice(kompetenzen_pool),
            "aufwand": round(random.uniform(1.0, 5.0), 1),
            "start": "03/2025",
            "ende": "06/2025"
        }
        for _ in range(random.randint(2, 4))
    ]
    projekte.append(projekt)
    teilaufgaben.extend(proj_ta)


In [5]:
# In CSV-Dateien schreiben
pd.DataFrame(personen).to_csv("personen.csv", index=False)
pd.DataFrame(projekte).to_csv("projekte.csv", index=False)
pd.DataFrame(teilaufgaben).to_csv("teilaufgaben.csv", index=False)

print("CSV-Dateien wurden erfolgreich erstellt: personen.csv, projekte.csv, teilaufgaben.csv")

CSV-Dateien wurden erfolgreich erstellt: personen.csv, projekte.csv, teilaufgaben.csv
